In [1]:
import os
import re
import time
import socket

In [2]:
# 크롤링 결과를 저장하는 파일
CRAWLED_RESULT_SAVED_FILE = "res/crawl_data/result.txt"

In [4]:
def record_url_info_thread():
    while (True):
        f = open(CRAWLED_RESULT_SAVED_FILE, 'w')
        print(f.write('test'))
        f.close()

In [5]:
#record_info()

4


In [8]:
!dir res\crawl_data

 C 드라이브의 볼륨: 오후반_OS
 볼륨 일련 번호: E808-E5FC

 C:\Users\user2\python_works\PythonWorkshop-1\res\crawl_data 디렉터리

2021-10-21  오후 03:45    <DIR>          .
2021-10-21  오후 03:45    <DIR>          ..
2021-10-21  오후 03:45                 4 result.txt
               1개 파일                   4 바이트
               2개 디렉터리  147,387,457,536 바이트 남음


In [9]:
!type res\crawl_data\result.txt

test


In [10]:
from flask import Flask, url_for, request, jsonify, redirect, Response
from flask_cors import CORS, cross_origin

import json
import requests

In [11]:
import selenium
from selenium import webdriver

In [51]:
# 키보드 입력 처리
from selenium.webdriver.common.keys import Keys

In [77]:
def cont_youtube_crawl_thread(text):
    cnt = 0
    
    #url = "https://www.youtube.com/"
    url = "https://www.youtube.com/results?search_query="

    driver = webdriver.Chrome(r"C:\chromedriver.exe")
    
    search_url = url + text
    driver.get(url = search_url)
    
    # body 정보 확보
    body = driver.find_element_by_tag_name('body')
    
    #hrefURL = []
    hrefURL = set()
    
    # 관련 영상 정보
    shelfContents = driver.find_elements_by_css_selector(
        "#contents > ytd-shelf-renderer > div > div > ytd-vertical-list-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
    )
    
    for item in shelfContents:
        if (item != ''):
            hrefURL.add(item.get_attribute('href'))
    
    # 무한 스크롤 하여 나머지 정보들 모두 확보
    #while (True):
    while cnt < 1000:
        #body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.END)
        
        cnt += 1
        print("cnt: ", cnt)
        
    allContents = driver.find_elements_by_css_selector(
        "#contents > ytd-item-section-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
    )

    for item in allContents:
        if (item != ''):
            hrefURL.add(item.get_attribute('href'))

    print(hrefURL)
    print("크롤링 개수: ", len(hrefURL))
    
    # y축으로 3000 픽셀 이동함
    #driver.execute_script("window.scrollTo(0, 3000)")

In [78]:
# 현재 상황에서 기록하는 스레드와 크롤링 하는 두 개의 스레드가 있음
# 여기에서 어떤 문제점이 발생할 수 있을까 ?

# 크롤링하는 스레드 - 검색 키워드를 기반으로 크롤링
# 기록하는 스레드 - 크롤링한 URL 저장

# 떡대(속도 느림, 수용량 높음) - 많은 양을 한 번에 모아서 이동
# 멸치(속도 빠름, 수용량 낮음) - 적은양이지만 중요한 정보를 빠르게 전송하는데 활용

In [80]:
cont_youtube_crawl_thread("시고르자브종")

clear


In [1]:
import threading
import time
import logging

In [4]:
def consume(cond):
    print("consume 스레드 시작")
    
    with cond:
        print("consume 스레드 대기")
        cond.wait()
        print("consume 스레드 리소스 사용 완료")

In [5]:
def produce(cond):
    print("produce 스레드 시작")
    
    with cond:
        print("produce 스레드 리소스 생성")
        print("produce 스레드 작업 재개 신호 발생")
        cond.notifyAll()

In [8]:
if __name__ == '__main__':
    cond = threading.Condition()
    con1 = threading.Thread(name = 'consumer1', target = consume, args = (cond,))
    con2 = threading.Thread(name = 'consumer2', target = consume, args = (cond,))
    prod = threading.Thread(name = 'producer1', target = produce, args = (cond,))
    
    con1.start()
    #time.sleep(5)
    con2.start()
    #time.sleep(5)
    prod.start()

consume 스레드 시작
consume 스레드 대기
consume 스레드 시작
consume 스레드 대기
produce 스레드 시작
produce 스레드 리소스 생성
produce 스레드 작업 재개 신호 발생
consume 스레드 리소스 사용 완료
consume 스레드 리소스 사용 완료


In [12]:
cnt = 1
print("cnt = {0}".format(cnt * 3))

cnt = 3


In [2]:
# A Thread는 2의 배수를 3개 모을때마다 정지
# B Thread는 3의 배수를 5개 모을때마다 정지
# 각 루프를 10번 돌면서 결과를 출력할 수 있게 만들어보자!
import threading
import time
import logging

def A_thread(cond):
    global A_flag
    global A_fin
    
    A_flag = False
    A_fin = False
    
    print("A 스레드 시작")
    cnt = 0
    
    for i in range(10):
        with cond:
            while True:
                cnt += 1
                
                print("{0}".format(cnt * 2))
                
                if (not(cnt % 3)):
                    break
                    
            time.sleep(1)
        
            print("A 스레드 대기")
            A_flag = True
            cond.wait()
            A_flag = False
            print("A 스레드 재시작")
            
    A_fin = True
    print("A Thread 종료")

def B_thread(cond):
    global B_flag
    global B_fin
    
    B_flag = False
    B_fin = False

    print("B 스레드 시작")
    cnt = 0
    
    for i in range(10):
        with cond:
            while True:
                cnt += 1
                
                print("{0}".format(cnt * 3))
                
                if (not(cnt % 5)):
                    break
                    
            time.sleep(1)
                      
            print("B 스레드 대기")
            B_flag = True
            cond.wait()
            B_flag = False
            print("B 스레드 재시작")
            
    B_fin = True
    print("B Thread 종료")
            
def manager(cond):
    print("manager 스레드 시작")
    
    while True:
        with cond:
            print("A_flag: {0}, B_flag: {1}".format(A_flag, B_flag))
            print("A_fin: {0}, B_fin: {1}".format(A_fin, B_fin))
            # 현재 A도 대기, B도 대기중인지 체크
            if (A_flag or B_flag):
                #print("A, B 스레드 작업 재개 신호 발생")
                cond.notifyAll()
                
            if (A_fin and B_fin):
                break
                
            time.sleep(1)
            
    print("Manager 종료")
        
if __name__ == '__main__':
    cond = threading.Condition()
    A = threading.Thread(name = 'A', target = A_thread, args = (cond,))
    B = threading.Thread(name = 'B', target = B_thread, args = (cond,))
    man = threading.Thread(name = 'manager', target = manager, args = (cond,))
    
    A.start()
    B.start()
    man.start()

A 스레드 시작
2
4
6
B 스레드 시작
manager 스레드 시작
A 스레드 대기
3
6
9
12
15
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A 스레드 재시작
8
10
12
A 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A 스레드 재시작
14
16
18
A 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A 스레드 재시작
20
22
24
A 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
18
21
24
27
30
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
33
36
39
42
45
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A 스레드 재시작
26
28
30
A 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
48
51
54
57
60
B 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
A 스레드 재시작
32
34
36
A 스레드 대기
A_flag: True, B_flag: True
A_fin: False, B_fin: False
B 스레드 재시작
63
66
69
72
75
B 스레드 대기
A_flag: 